## firebase setup

In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import os

SERVICE_ACCOUNT_KEY_PATH = os.environ.get("FIREBASE_SERVICE_ACCOUNT_KEY_PATH", "./cstam2-1f2ec-firebase-adminsdk-fbsvc-2ab61a7ed6.json")

try:
    # Check if the default app is already initialized
    app = firebase_admin.get_app()
    print("Firebase Admin SDK already initialized. Reusing existing app instance.")
except ValueError:
    # If not initialized, proceed with initialization
    try:
        cred = credentials.Certificate(SERVICE_ACCOUNT_KEY_PATH)
        app = firebase_admin.initialize_app(cred)
        print("Firebase Admin SDK initialized successfully.")
    except Exception as e:
        print(f"Error during Firebase Admin SDK initialization: {e}")
        # It's crucial to handle this error, as your app can't write to Firestore without it.
        raise # Re-raise to stop the script if Firebase initialization fails

db = firestore.client(app=app) 


An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'
Firebase Admin SDK initialized successfully.


/usr/local/lib/python3.8/dist-packages/google/api_core/_python_version_support.py:237: FutureWarning: You are using a non-supported Python version (3.8.10). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)


## setup spark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, to_timestamp, from_unixtime
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, DoubleType
import time
import logging
# ---------- CONFIG ----------
KAFKA_BOOTSTRAP = "kafka:29092"                  

CHECKPOINT_BASE = "/data/checkpoints"

# Configure logger
logger = logging.getLogger("DataCleaning")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter("[%(asctime)s] %(levelname)s - %(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
# ---------- Spark session ----------
spark = (
    SparkSession.builder
    .appName("health-streams-to-firebase")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")

# ---------- utils ----------
def read_topic(topic, schema):
    return (
        spark.readStream
             .format("kafka")
             .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP)
             .option("subscribe", topic)
             .option("startingOffsets", "latest")
             .load()
             .selectExpr("CAST(value AS STRING) as json_str")
             .select(from_json(col("json_str"), schema).alias("data"))   # <<< fixed
             .select("data.*")
    )


heart_rate_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("time", TimestampType(), True),
    StructField("heart_rate", IntegerType(), True)
])
steps_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("time", TimestampType(), True),
    StructField("steps", IntegerType(), True)
])
calories_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("time", TimestampType(), True),
    StructField("calories", DoubleType(), True)
])

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/10 10:41:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## cleaning functions

In [3]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.functions import current_timestamp

## ---------- timestamp normalization & cleaning ----------
def normalize_timestamp(df):
    candidates = [c for c in ["timestamp", "ts", "ts_ms", "time", "created_at"] if c in df.columns]
    logger.info(f"Normalizing timestamp — candidates: {candidates}")

    if not candidates:
        logger.warning("No timestamp found — creating current-time column.")
        return df.withColumn("timestamp", current_timestamp())

    src = candidates[0]
    logger.info(f"Using '{src}' as timestamp source.")
    src_col = col(src)

    # Only convert numeric timestamps; keep native timestamps unchanged
    df = df.withColumn(
        "timestamp",
        when(src_col.cast(DoubleType()).isNotNull(), 
             to_timestamp(from_unixtime(
                 (src_col.cast(DoubleType()) / when(src_col.cast(DoubleType()) > 1e12, lit(1000.0)).otherwise(lit(1.0))).cast("long")
             ))
        ).otherwise(src_col.cast(TimestampType()))
    )

    return df.drop(src)
def clean_heart_rate(df):
    logger.info("Cleaning heart rate stream...")
    df = df.withColumn("heart_rate", col("heart_rate").cast(IntegerType()))
    df = normalize_timestamp(df)
    df = df.dropna(subset=["heart_rate", "timestamp"])
    df = df.filter((col("heart_rate") > 0) & (col("heart_rate") < 300))
    logger.info("Heart rate cleaning complete (streaming mode).")
    return df


def clean_calories(df):
    logger.info("Cleaning calories stream...")
    df = df.withColumn("calories", col("calories").cast(DoubleType()))
    df = normalize_timestamp(df)
    df = df.dropna(subset=["calories", "timestamp"])
    df = df.filter(col("calories") >= 0)
    logger.info("Calories cleaning complete (streaming mode).")
    return df


def clean_steps(df):
    logger.info("Cleaning steps stream...")
    df = df.withColumn("steps", col("steps").cast(IntegerType()))
    df = normalize_timestamp(df)
    df = df.dropna(subset=["steps", "timestamp"])
    df = df.filter(col("steps") >= 0)
    logger.info("Steps cleaning complete (streaming mode).")
    return df
    

  
# ---------- prepare streams ----------

heart_rate_df = read_topic("heartrate", heart_rate_schema)
calories_df = read_topic("calories", calories_schema)
steps_df = read_topic("steps", steps_schema)

hr_clean = clean_heart_rate(heart_rate_df)
cal_clean = clean_calories(calories_df)
st_clean = clean_steps(steps_df)


[2025-11-10 10:41:56,839] INFO - Cleaning heart rate stream...
[2025-11-10 10:41:56,876] INFO - Normalizing timestamp — candidates: ['time']
[2025-11-10 10:41:56,877] INFO - Using 'time' as timestamp source.
[2025-11-10 10:41:56,946] INFO - Heart rate cleaning complete (streaming mode).
[2025-11-10 10:41:56,946] INFO - Cleaning calories stream...
[2025-11-10 10:41:56,963] INFO - Normalizing timestamp — candidates: ['time']
[2025-11-10 10:41:56,964] INFO - Using 'time' as timestamp source.
[2025-11-10 10:41:57,029] INFO - Calories cleaning complete (streaming mode).
[2025-11-10 10:41:57,030] INFO - Cleaning steps stream...
[2025-11-10 10:41:57,052] INFO - Normalizing timestamp — candidates: ['time']
[2025-11-10 10:41:57,053] INFO - Using 'time' as timestamp source.
[2025-11-10 10:41:57,187] INFO - Steps cleaning complete (streaming mode).


## anomaly detection functions

In [4]:
import time
from datetime import datetime

def send_alert_to_firestore(user_id, alert_type, value, firestore_client,timestamp):
    """
    Write an anomaly alert to Firestore.
    Each alert becomes a document in the 'alerts' collection.

    Args:
        user_id (str): The ID of the affected user.
        alert_type (str): Type of the alert, e.g., 'heart_rate', 'calories', etc.
        firestore_client: Initialized Firestore client (google.cloud.firestore.Client).
    """
    try:
        alerts_ref = firestore_client.collection("alerts")
        alert_doc = {
            "user_id": user_id,
            "type": alert_type,
            "value": value,
            "detected_at": timestamp,
            "alert_created_at_unix": int(time.time()),
            "status":"waiting"
        }

        alerts_ref.add(alert_doc)
        print(f"[ok] Alert written for user={user_id}: {alert_type} — {value}")

    except Exception as e:
        print(f"[error] Failed to write alert for user={user_id}: {e}")
        
def detect_heart_rate_anomalies(df, firestore_client):
    anomalies = df.filter((col("heart_rate") > 180) | (col("heart_rate") < 45))
    if anomalies.rdd.isEmpty():
        return
    for row in anomalies.toLocalIterator():
        send_alert_to_firestore(
            user_id=row.user_id,
            alert_type="heart_rate",
            value=row.heart_rate,
            firestore_client=firestore_client,
            timestamp= row.timestamp
        )
        
def detect_low_calories(df, firestore_client):
    low_activity = df.filter(col("calories") < 0.5)  # e.g. less than 0.5 kcal/min or similar
    if low_activity.rdd.isEmpty():
        return
    for row in low_activity.toLocalIterator():
        send_alert_to_firestore(
            user_id=row.user_id,
            alert_type="low_calories",
            value=row.calories,
            firestore_client=firestore_client,
            timestamp= row.timestamp
        )
        
def detect_inactivity(df, firestore_client):
    inactive = df.filter(col("steps") == 0)
    if inactive.rdd.isEmpty():
        return
    for row in inactive.toLocalIterator():
        send_alert_to_firestore(
            user_id=row.user_id,
            alert_type="inactivity",
            value=0,
            firestore_client=firestore_client,
            timestamp= row.timestamp
        )


## write stream functions

In [5]:
import logging
import time
import datetime
import decimal
import json
from google.api_core.exceptions import GoogleAPICallError, RetryError

# logger (reuse your existing logger or this)
fs_logger = logging.getLogger("firestore-writer")
fs_logger.setLevel(logging.INFO)
if not fs_logger.hasHandlers():
    h = logging.StreamHandler()
    h.setFormatter(logging.Formatter("[%(levelname)s] %(asctime)s - %(message)s"))
    fs_logger.addHandler(h)


def _serialize_value(v):
    """Make values Firestore-safe: datetime -> ISO, Decimal -> float, fallback to str for non-JSON."""
    if v is None:
        return None
    if isinstance(v, (datetime.datetime, datetime.date)):
        return v.isoformat()
    if isinstance(v, decimal.Decimal):
        return float(v)
    # Bytes -> decode
    if isinstance(v, (bytes, bytearray)):
        try:
            return v.decode()
        except Exception:
            return str(v)
    # Test JSON-serializable quickly
    try:
        json.dumps(v)
        return v
    except Exception:
        return str(v)


def _commit_with_retries(batch_obj, max_retries=3, base_backoff=0.5):
    """Commit a Firestore batch with retries on transient errors."""
    attempt = 0
    while True:
        try:
            batch_obj.commit()
            return
        except (GoogleAPICallError, RetryError, IOError) as e:
            attempt += 1
            if attempt > max_retries:
                fs_logger.exception("Firestore commit failed after %d attempts", attempt - 1)
                raise
            backoff = base_backoff * (2 ** (attempt - 1))
            fs_logger.warning("Transient error committing firestore batch (attempt %d). Backing off %.2fs. Error: %s",
                              attempt, backoff, str(e))
            time.sleep(backoff)


def make_firestore_writer(collection_name, firestore_client, batch_size=500, max_retries=3):
    """
    Writes to Firestore under this structure:
      users/{user_id}/{collection_name}/{auto_doc_id}

    collection_name: e.g. "heart_rate", "calories", "steps"
    """
    def write_batch_to_firestore(batch_df, epoch_id):
        epoch_str = str(epoch_id) if epoch_id is not None else "(no-epoch)"
        try:
            rows = batch_df.count()
        except Exception as e:
            fs_logger.exception("[epoch %s] failed to count batch: %s", epoch_str, e)
            rows = None

        if not rows:
            fs_logger.info("[epoch %s] empty, skipping collection=%s", epoch_str, collection_name)
            return

        fs_logger.info("[epoch %s] writing %s rows to Firestore subcollections '%s'", epoch_str, rows, collection_name)

        docs_written = 0
        ops_in_current_batch = 0
        fs_batch = firestore_client.batch()

        try:
            for row in batch_df.toLocalIterator():
                data = row.asDict(recursive=True)
                user_id = data.get("user_id")

                if not user_id:
                    fs_logger.warning("[epoch %s] skipping row without user_id: %s", epoch_str, data)
                    continue

                # Serialize all values to Firestore-safe types
                for k, v in list(data.items()):
                    data[k] = _serialize_value(v)

                # Path: users/{user_id}/{collection_name}/{auto_doc_id}
                doc_ref = (
                    firestore_client.collection("users")
                    .document(str(user_id))
                    .collection(collection_name)
                    .document()
                )

                fs_batch.set(doc_ref, data)
                ops_in_current_batch += 1

                if ops_in_current_batch >= batch_size:
                    _commit_with_retries(fs_batch, max_retries=max_retries)
                    docs_written += ops_in_current_batch
                    ops_in_current_batch = 0
                    fs_batch = firestore_client.batch()

            if ops_in_current_batch > 0:
                _commit_with_retries(fs_batch, max_retries=max_retries)
                docs_written += ops_in_current_batch

            fs_logger.info("[epoch %s] wrote %d docs under users/*/%s/", epoch_str, docs_written, collection_name)

        except Exception as e:
            fs_logger.exception("[epoch %s] Firestore write failed: %s", epoch_str, e)
            raise

    return write_batch_to_firestore


## cleaning stream queries

In [ ]:
# start streams separately with their own checkpoints
# Pass the initialized Firestore client 'db' to your writer functions
hr_query = (
    hr_clean
    .writeStream
    .foreachBatch(make_firestore_writer("heart_rate", db)) # Pass the Firestore client here
    .option("checkpointLocation", f"{CHECKPOINT_BASE}/heart_rate")
    .outputMode("append")
    .start()
)

cal_query = (
    cal_clean
    .writeStream
    .foreachBatch(make_firestore_writer("calories", db)) # Pass the Firestore client here
    .option("checkpointLocation", f"{CHECKPOINT_BASE}/calories")
    .outputMode("append")
    .start()
)

st_query = (
    st_clean
    .writeStream
    .foreachBatch(make_firestore_writer("steps", db)) # Pass the Firestore client here
    .option("checkpointLocation", f"{CHECKPOINT_BASE}/steps")
    .outputMode("append")
    .start()
)
# Start streams for each detection type
hr_alerts = (
    hr_clean.writeStream
    .foreachBatch(lambda batch_df, epoch_id: detect_heart_rate_anomalies(batch_df, db))
    .option("checkpointLocation", f"{CHECKPOINT_BASE}/alerts_hr")
    .start()
)

cal_alerts = (
    cal_clean.writeStream
    .foreachBatch(lambda batch_df, epoch_id: detect_low_calories(batch_df, db))
    .option("checkpointLocation", f"{CHECKPOINT_BASE}/alerts_calories")
    .start()
)

st_alerts = (
    st_clean.writeStream
    .foreachBatch(lambda batch_df, epoch_id: detect_inactivity(batch_df, db))
    .option("checkpointLocation", f"{CHECKPOINT_BASE}/alerts_steps")
    .start()
)

spark.streams.awaitAnyTermination()


print("Started all streams. Waiting...")
spark.streams.awaitAnyTermination()

[INFO] 2025-11-10 10:42:06,414 - [epoch 239] writing 1 rows to Firestore subcollections 'calories'
[INFO] 2025-11-10 10:42:06,414 - [epoch 239] writing 1 rows to Firestore subcollections 'steps'
[INFO] 2025-11-10 10:42:06,415 - [epoch 13535] writing 12 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 10:42:07,880 - [epoch 13535] wrote 12 docs under users/*/heart_rate/
[INFO] 2025-11-10 10:42:08,039 - [epoch 239] wrote 1 docs under users/*/steps/
[INFO] 2025-11-10 10:42:08,041 - [epoch 239] wrote 1 docs under users/*/calories/


[ok] Alert written for user=1503960366: inactivity — 0


[INFO] 2025-11-10 10:42:08,523 - [epoch 13536] writing 5 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 10:42:09,312 - [epoch 13536] wrote 5 docs under users/*/heart_rate/
[INFO] 2025-11-10 10:42:09,543 - [epoch 13537] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 10:42:09,764 - [epoch 13537] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 10:42:10,443 - [epoch 13538] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 10:42:10,705 - [epoch 13538] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 10:42:11,341 - [epoch 13539] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 10:42:11,695 - [epoch 13539] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 10:42:12,321 - [epoch 13540] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 10:42:12,739 - [epoch 13540] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 10:42:13,351 - [epoch 13541] writing 

[ok] Alert written for user=8378563200: inactivity — 0


[INFO] 2025-11-10 10:43:01,955 - [epoch 13589] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 10:43:02,578 - [epoch 13590] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 10:43:02,964 - [epoch 13590] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 10:43:03,570 - [epoch 13591] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 10:43:03,983 - [epoch 13591] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 10:43:04,588 - [epoch 13592] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 10:43:05,004 - [epoch 13592] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 10:43:05,604 - [epoch 13593] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 10:43:05,975 - [epoch 13593] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 10:43:06,606 - [epoch 13594] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 10:43:07,005 - [epoch 13594] wrote 1 

...

[INFO] 2025-11-10 11:09:01,868 - [epoch 15135] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:09:02,422 - [epoch 15136] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:09:02,861 - [epoch 15136] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:09:03,420 - [epoch 15137] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:09:03,898 - [epoch 15137] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:09:04,454 - [epoch 15138] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:09:04,861 - [epoch 15138] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:09:05,421 - [epoch 15139] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:09:05,870 - [epoch 15139] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:09:06,432 - [epoch 15140] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:09:06,880 - [epoch 15140] wrote 1 

[ok] Alert written for user=8583815059: inactivity — 0


[INFO] 2025-11-10 11:10:01,707 - [epoch 15195] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:10:02,151 - [epoch 15195] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:10:02,710 - [epoch 15196] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:10:03,165 - [epoch 15196] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:10:03,718 - [epoch 15197] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:10:04,160 - [epoch 15197] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:10:04,714 - [epoch 15198] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:10:05,163 - [epoch 15198] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:10:05,719 - [epoch 15199] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:10:06,177 - [epoch 15199] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:10:06,730 - [epoch 15200] writing 

[ok] Alert written for user=5577150313: inactivity — 0


[INFO] 2025-11-10 11:11:01,953 - [epoch 15255] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:11:02,402 - [epoch 15255] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:11:02,962 - [epoch 15256] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:11:03,422 - [epoch 15256] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:11:03,998 - [epoch 15257] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:11:04,446 - [epoch 15257] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:11:05,018 - [epoch 15258] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:11:05,421 - [epoch 15258] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:11:05,981 - [epoch 15259] writing 1 rows to Firestore subcollections 'heart_rate'
[INFO] 2025-11-10 11:11:06,531 - [epoch 15259] wrote 1 docs under users/*/heart_rate/
[INFO] 2025-11-10 11:11:07,088 - [epoch 15260] writing 

In [ ]:
for q in spark.streams.active:
    q.stop()
print("✅ Stopped all active queries.")
